In [1]:
import csv 
import numpy as np
from numpy.linalg import inv #矩阵求逆
import random
import math
import sys
import pandas as pd

In [2]:
file = open('data/train.csv', 'r',encoding='big5')
read_data = csv.reader(file,delimiter=',')

data = []
for i in range(18):
    data.append([])
idx = 0
for row in read_data:
    if idx!=0:
        for i in range(3,27):
            if row[i]!='NR':
                data[(idx-1)%18].append(float(row[i]))
            else:
                data[(idx-1)%18].append(float(0))
    idx+=1
file.close()

In [3]:
x = []
y = []
for i in range(12):
    for j in range(471):
        x.append([])
        for t in range(18):
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s])
        y.append(data[9][480*i+j+9])

x,y = np.array(x), np.array(y)
x = np.concatenate((np.ones((x.shape[0],1)),x),axis=1)

In [4]:
file = open('data/test.csv', 'r',encoding='big5')
read_data = csv.reader(file, delimiter=',')
idx = 0
test_data = []

for row in read_data:
    if idx%18==0:
        test_data.append([])
        for i in range(2,11):
            if row[i]!='NR':
                test_data[idx//18].append(float(row[i]))
            else:
                test_data[idx//18].append(float(0))
    else:
        for i in range(2,11):
            if row[i]!='NR':
                test_data[idx//18].append(float(row[i]))
            else:
                test_data[idx//18].append(float(0))
    idx+=1
test_data=np.array(test_data)

test_x = np.concatenate((np.ones((test_data.shape[0],1)),test_data),axis=1)

In [5]:
x.shape,test_x.shape,y.shape

((5652, 163), (240, 163), (5652,))

In [11]:
learning_rate = 1
epochs = 10000
eps = 1e-3
seed = 2018
cost = []
s =  np.zeros((x.shape[1],1))

In [ ]:
gen = np.random.RandomState(seed)
w = gen.normal(loc=0.0, scale=1e-3, size=(x.shape[1],1))

for ep in range(epochs):
    pred = x@w
    loss = pred - y.reshape((-1,1))
    cost = np.sum(loss**2)/x.shape[0]
    rmse_cost = np.sqrt(cost)
    #adagrad
    grad = 2*x.T@loss
    s += grad**2
    g_ = learning_rate/np.sqrt(eps+s) * grad
    w = w-g_
    print('epoch:{},|rmse:{}'.format(ep,rmse_cost))  #final 5.89

In [15]:
np.save('model.npy',w)
w = np.load('model.npy')

In [25]:
def get_err(file1, file2):
    y1 = pd.read_csv(file1).value.values
    y2 = pd.read_csv(file2).value.values
    y1,y2 = np.array(y1), np.array(y2)
    errors = np.sqrt(np.sum((y1-y2)**2)/len(y1))
    print(errors)

In [78]:
ans = []
w = np.array(w).squeeze()   #专程163个数字，否则保存的是【】
for i in range(len(test_x)):
    ans.append(["id_"+str(i)])
    a = test_x[i]@w
    ans[i].append(a)

In [79]:
text = open('to_submission.csv','w+')
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()

In [80]:
get_err('to_submission.csv','ans.csv')

6.767832088956983
